In [1]:
import paddle.fluid as fluid
import numpy as np
from paddle.fluid.debugger import draw_block_graphviz

In [2]:
test_data = np.arange(9).reshape(3,3).astype('float32')
test_data

array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]], dtype=float32)

## Create Variable in Program

In [3]:
main = fluid.Program()
block = main.current_block()

In [4]:
x = block.create_var(name="x", shape=[3,3], dtype='float32')
x, type(x)

(name: "x"
 type {
   type: LOD_TENSOR
   lod_tensor {
     tensor {
       data_type: FP32
       dims: 3
       dims: 3
     }
   }
 }, paddle.fluid.framework.Variable)

In [5]:
with fluid.program_guard(main):
    y = fluid.layers.ops.mean(x) # Layer会向当前Main program和Starup program插入op
y, type(y)

(name: "mean_0.tmp_0"
 type {
   type: LOD_TENSOR
   lod_tensor {
     tensor {
       data_type: FP32
       dims: 1
     }
   }
 }
 persistable: false, paddle.fluid.framework.Variable)

In [6]:
exe = fluid.executor.Executor(fluid.CPUPlace())
# 这里不需要startup program，因为全部变量都创建了

In [7]:
exe.run(program=main, fetch_list=[y], feed={x.name: test_data})

[array([4.], dtype=float32)]

In [8]:
draw_block_graphviz(y.block, path="out.pdf")

## Create Variable in global block

In [9]:
x = fluid.layers.create_global_var(shape=[3,3], value=1.0, dtype='float32')

In [10]:
y = fluid.layers.ops.mean(x)

In [11]:
exe = fluid.executor.Executor(fluid.CPUPlace())

In [12]:
exe.run(fetch_list=[y], feed={x.name: test_data})

[array([4.], dtype=float32)]